# Description

For defining functional groups and reaction mechanisms for polymerization

# Core Imports

In [ ]:
# Custom Imports
from polysaccharide.extratypes import ResidueSmarts

from polysaccharide.molutils import reactions
from polysaccharide.molutils.rdmol.rdtypes import *
from polysaccharide.molutils.rdmol import rdkdraw, rdprops, rdbond, rdlabels

# File I/O
from pathlib import Path
import json

# Cheminformatics
from rdkit import Chem
from rdkit.Chem import rdChemReactions

# Static Paths
RXN_FILES_PATH = Path('rxn_smarts')
RXN_FILES_PATH.mkdir(exist_ok=True)

## Defining target functional groups

In [ ]:
rdkdraw.set_rdkdraw_size(200, 5)

fn_group_smiles_base = {
    'amine'          : '[NH3]',
    'cyclocarbonate' : 'C1OC(=O)OC1',
    'hydroxyl'       : '[OH2]',
    'isocyanate'     : 'O=C=N'
}

fn_group_smarts_portless = {}
for group_name, smiles in fn_group_smiles_base.items():
    rdmol = Chem.MolFromSmiles(smiles)
    rdmol = Chem.AddHs(rdmol)
    rdmol = rdlabels.assign_ordered_atom_map_nums(rdmol, in_place=False)
    display(rdmol)

    fn_group_smarts_portless[group_name] = Chem.MolToSmarts(rdmol)

In [ ]:
fn_group_smarts = { # add ports
    'amine'          : '[#7D3:1](-[#1:2])(-[#1:3])-[*:4]',
    'cyclocarbonate' : '[#6:1]1(-[#8:2]-[#6:3](=[#8:4])-[#8:5]-[#6:6]-1(-[#1:9])-[*:10])(-[#1:7])-[*:8]',
    'hydroxyl'       : '[#8:1](-[#1:2])-[*:3]',
    'isocyanate'     : '[#8:1]=[#6:2]=[#7D2:3]-[*:4]'
}

with (RXN_FILES_PATH / 'fn_group_smarts.json').open('w') as file:
    json.dump(fn_group_smarts, file, indent=4)

fn_groups = {}
for group, smarts in fn_group_smarts.items():
    group_mol = Chem.MolFromSmarts(smarts)
    display(group_mol)
    fn_groups[group] = group_mol

# Defining polymerization reactions

In [ ]:
rdkdraw.set_rdkdraw_size(400, 3/2)
rxns, rxn_smarts = {}, {}

## NIPU reaction

In [ ]:
# defining targets
reactive_groups = (
    fn_groups['cyclocarbonate'],
    fn_groups['amine']
)
chemistry = 'NIPU'

bond_break_map_nums = (
    (11, 12),
    (2, 3)
)

bond_make_map_nums = (
    (11, 3),
    (2, 12)
)

# 1) extracting and labelling reactants
reactant1, reactant2 = rdlabels.assign_contiguous_atom_map_nums(*reactive_groups, in_place=False)
reactant1_SMARTS = Chem.MolToSmarts(reactant1) 
reactant2_SMARTS = Chem.MolToSmarts(reactant2) 
reactants = Chem.MolFromSmarts(f'{reactant1_SMARTS}.{reactant2_SMARTS}')
display(reactants)

# 2) cleaving broken bonds
product = Chem.RWMol(reactants) # create editable Mol
for atom_pair in bond_break_map_nums:
    rdbond.decrease_bond_order(
        product,
        *rdlabels.atom_ids_by_map_nums(product, *atom_pair),
        dummyLabels=True,
        in_place=True
    )
display(product)

# 3) forming new bonds with cleaved groups
for atom_pair in bond_make_map_nums:
    rdbond.increase_bond_order(
        product,
        *rdlabels.atom_ids_by_map_nums(product, *atom_pair),
        prioritize_unlabelled_ports=True,
        in_place=True
    )
rdlabels.clear_atom_isotopes(product, in_place=True)
display(product)

# defining reaction
rxn = reactions.AnnotatedReaction.from_rdmols(reactant_templates=[reactant1, reactant2], product_templates=[product])
rxns[chemistry] = rxn
rxn_smarts[chemistry] = rdChemReactions.ReactionToSmarts(rxn)

## Urethane reaction

In [ ]:
# defining targets
reactive_groups = (
    fn_groups['isocyanate'],
    fn_groups['hydroxyl']
)
chemistry = 'urethane'

bond_break_map_nums = (
    (5, 6),
    (3, 2)
)

bond_make_map_nums = (
    (3, 6),
    (5, 2)
)

# 1) extracting and labelling reactants
reactant1, reactant2 = rdlabels.assign_contiguous_atom_map_nums(*reactive_groups, in_place=False)
reactant1_SMARTS = Chem.MolToSmarts(reactant1) 
reactant2_SMARTS = Chem.MolToSmarts(reactant2) 
reactants = Chem.MolFromSmarts(f'{reactant1_SMARTS}.{reactant2_SMARTS}')
display(reactants)

# 2) cleaving broken bonds
product = Chem.RWMol(reactants) # create editable Mol
for atom_pair in bond_break_map_nums:
    rdbond.decrease_bond_order(
        product,
        *rdlabels.atom_ids_by_map_nums(product, *atom_pair),
        dummyLabels=True,
        in_place=True
    )
display(product)

# 3) forming new bonds with cleaved groups
for atom_pair in bond_make_map_nums:
    rdbond.increase_bond_order(
        product,
        *rdlabels.atom_ids_by_map_nums(product, *atom_pair),
        prioritize_unlabelled_ports=True,
        in_place=True
    )
rdlabels.clear_atom_isotopes(product, in_place=True)
display(product)

# defining reaction
rxn = reactions.AnnotatedReaction.from_rdmols(reactant_templates=[reactant1, reactant2], product_templates=[product])
rxns[chemistry] = rxn
rxn_smarts[chemistry] = rdChemReactions.ReactionToSmarts(rxn)

## Saving SMARTS-based reaction equations to file

In [ ]:
with (RXN_FILES_PATH / 'rxn_smarts.json').open('w') as file:
    json.dump(rxn_smarts, file, indent=4)

for chemistry, rxn in rxns.items():
    rxn.to_rxnfile(RXN_FILES_PATH / f'{chemistry}.rxn')